references: 
1. https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html
1. https://towardsdatascience.com/how-to-set-up-a-machine-learning-model-for-legal-contract-review-fe3b48b05a0e
1. https://towardsdatascience.com/how-to-set-up-a-machine-learning-model-for-legal-contract-review-part-2-6ecbbe680ba
1. https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
1. https://arxiv.org/abs/2103.06268
1. https://www.atticusprojectai.org/
1. https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html

In [ ]:
!git clone https://github.com/TheAtticusProject/cuad.git

In [ ]:
%pip install streamlit
%pip install st-annotated-text

In [ ]:
import streamlit as st
#from st_annotated_text import annotated_text

In [ ]:
# get checkpoints - https://zenodo.org/record/4599830
!wget https://zenodo.org/record/4599830/files/roberta-base.zip  # roberta-base

In [ ]:
# OPTIONAL - get checkpoints - https://zenodo.org/record/4599830
!wget https://zenodo.org/record/4599830/files/deberta-v2-xlarge.zip  # deberta

In [ ]:
# OPTIONAL - get checkpoints - https://zenodo.org/record/4599830
!wget https://zenodo.org/record/4599830/files/roberta-large.zip  # roberta-large

In [ ]:
!mkdir /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/data

In [ ]:
!unzip /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/data.zip -d /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/data

In [ ]:
!mkdir /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/trained_models

In [ ]:
!unzip /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/roberta-base.zip -d /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/trained_models

In [ ]:
!python /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/evaluate.py

In [ ]:
%cd /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/

In [1]:
!pwd

/home/ec2-user/SageMaker/LegalContractAnalysis/cuad


In [2]:
!pip install tqdm

     |████████████████████████████████| 76 kB 6.0 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 54.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install tensorboard

     |████████████████████████████████| 5.6 MB 36.7 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 152 kB 41.8 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 64.4 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 59.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 36.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 54.9 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 57.1 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
!python train.py \
        --output_dir ./train_models/roberta-base \
        --model_type roberta \
        --model_name_or_path /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/trained_models/roberta-base/ \
        --data_dir ./data/ \
        --predict_file /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/data/test.json \
        --do_eval \
        --version_2_with_negative \
        --per_gpu_eval_batch_size=40  \
        --max_seq_length 512 \
        --max_answer_length 512 \
        --doc_stride 256 \
        --save_steps 1000 \
        --n_best_size 20 \
        --cache_dir /home/ec2-user/SageMaker/LegalContractAnalysis/cuad/cache/ \
        --overwrite_output_dir \
        --threads 4

In [5]:
import torch
import transformers

In [33]:
!pwd

/home/ec2-user/SageMaker/LegalContractAnalysis/cuad


## Load Model and Tokenizer

Note that the Question Answering Models are not yet compatible with fast tokenizer so we set the flag to False

In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("./trained_models/roberta-base/")
model = AutoModelForQuestionAnswering.from_pretrained("./trained_models/roberta-base/")

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [8]:
tokenizer("how much wood can a woodchuck chuck")

{'input_ids': [0, 9178, 203, 5627, 64, 10, 5627, 611, 5858, 34600, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# Create the Question / Context

To perform question answering on the contract we need to concatenate the question with the contract data.  Next the data is tokenized (convert from words to numbers) and then fed to the trained model.  The model will then provide two outputs - The start and end logits

In [14]:
def single_prediction(question,context):
    
    # 1. TOKENIZE THE INPUT
    # note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for 
    # exploration but you cannot feed that into a model. 
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

    # 2. OBTAIN MODEL SCORES
    # the AutoModelForQuestionAnswering class includes a span predictor on top of the model. 
    # the model returns answer start and end scores for each word in the text
    output = model(**inputs)
    answer_start_scores, answer_end_scores = model(**inputs,return_dict=False)
    answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

    # 3. GET THE ANSWER SPAN
    # once we have the most likely start and end tokens, we grab all the tokens between them
    # and convert tokens back to words!
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

    print(f'Question:  {question}')
    print()
    print(f'Answer:  {answer}')

#### Example 1

In [15]:
context = """Exhibit 10.2\n\n______________________________________________________________________________\n\nCO-PROMOTION AGREEMENT\n\nby and between\n\nDOVA PHARMACEUTICALS, INC.\n\nand\n\nVALEANT PHARMACEUTICALS NORTH AMERICA LLC\n\nSeptember 26, 2018\n\n______________________________________________________________________________\n\nCONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nTABLE OF CONTENTS\n\nPage\n\nARTICLE 1 DEFINITIONS 1\n\nARTICLE 2 RIGHTS AND OBLIGATIONS 8\n\n2.1 Engagement; Grant of Rights. 8\n\n2.2 Retention of Rights. 9\n\n2.3 Non-Competition; Non-Solicitation. 9\n\n2.4 Dova Trademarks and Copyrights. 10\n\nARTICLE 3 JOINT STEERING COMMITTEE 11\n\n3.1 Formation of the JSC. 11\n\n3.2 Meetings and Minutes. 11\n\n3.3 Purpose of the JSC. 11\n\n3.4 Decision Making. 13\n\n3.5 Marketing Sub-Committee. 13\n\n"""

print(context)

Exhibit 10.2

______________________________________________________________________________

CO-PROMOTION AGREEMENT

by and between

DOVA PHARMACEUTICALS, INC.

and

VALEANT PHARMACEUTICALS NORTH AMERICA LLC

September 26, 2018

______________________________________________________________________________

CONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.

Source: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018





TABLE OF CONTENTS

Page

ARTICLE 1 DEFINITIONS 1

ARTICLE 2 RIGHTS AND OBLIGATIONS 8

2.1 Engagement; Grant of Rights. 8

2.2 Retention of Rights. 9

2.3 Non-Competition; Non-Solicitation. 9

2.4 Dova Trademarks and Copyrights. 10

ARTICLE 3 JOINT STEERING COMMITTEE 11

3.1 Formation of the JSC. 11

3.2 Meetings and Minutes. 11

3.3 Purp

In [16]:
question = "Highlight the parts (if any) of this contract related to \"Agreement Date\" that should be reviewed by a lawyer. Details: The date of the contract"

In [17]:
single_prediction(question,context)

Question:  Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract

Answer:  September 26, 2018


#### Example 2

In [10]:
context = """10.	SUSPENSION:  

(a)  If Licensor breaches or is in default of any of its representations and warranties hereunder, Fox-Warner, in addition to any and all other rights it may have under this Agreement or in law or in equity, may suspend its distribution hereunder, and any payments which might otherwise be due Licensor until such occurrence is ended or remedied, and the Term of this Agreement shall be extended for a period equal to the length of such suspension.  Before Fox-Warner can declare a breach or default under this provision, it shall give Licensor written notice specifying the nature of the breach, and Licensor shall have thirty (30) days within which to effect a cure of such alleged breach.

(b)  Producer expressly understands and agrees that if Fox-Warner breaches this Agreement, including, without limitation, a breach of its obli¬gations to make payments and provide statements hereunder, the damage, if any, caused Producer thereby will not be irreparable or otherwise suf¬ficient to entitle Producer to injunctive or other equit¬able relief.  Producer agrees that the rights and remedies available to Producer in any such event shall be strictly limited to the right, if any, to recover damages in an action at law, and Producer shall not be entitled by reason of any such breach to terminate or rescind this Agreement, to restrain Fox-Warner's exercise of any of Fox-Warner's rights here¬under, or to restrain the exhibition or exploitation of the Picture.  Before Licensor can declare a breach under this provision, it shall give Fox-Warner written notice specifying the nature of the breach, and Fox-Warner shall have thirty (30) days within which to effect a cure of such alleged breach.
"""

print(context)

10.	SUSPENSION:  

(a)  If Licensor breaches or is in default of any of its representations and warranties hereunder, Fox-Warner, in addition to any and all other rights it may have under this Agreement or in law or in equity, may suspend its distribution hereunder, and any payments which might otherwise be due Licensor until such occurrence is ended or remedied, and the Term of this Agreement shall be extended for a period equal to the length of such suspension.  Before Fox-Warner can declare a breach or default under this provision, it shall give Licensor written notice specifying the nature of the breach, and Licensor shall have thirty (30) days within which to effect a cure of such alleged breach.

(b)  Producer expressly understands and agrees that if Fox-Warner breaches this Agreement, including, without limitation, a breach of its obli¬gations to make payments and provide statements hereunder, the damage, if any, caused Producer thereby will not be irreparable or otherwise suf¬f

In [11]:
question = 'Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.'

In [12]:
single_prediction(question,context)

Question:  Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.

Answer:   Producer agrees that the rights and remedies available to Producer in any such event shall be strictly limited to the right, if any, to recover damages in an action at law, and Producer shall not be entitled by reason of any such breach to terminate or rescind this Agreement, to restrain Fox-Warner's exercise of any of Fox-Warner's rights here¬under, or to restrain the exhibition or exploitation of the Picture.


#### Example 3

In [ ]:
context = """14.	FURTHER ASSURANCES:  Licensor shall execute and deliver to Fox-Warner promptly upon Fox-Warner's request, any other instruments or documents considered by Fox-Warner to be necessary or desirable to evidence, effectuate or confirm this Agreement, or any of the terms and conditions hereof.  Licensor shall provide Fox-Warner with a Laboratory Access Letter and a Transfer of Rights, substantially in the form of Exhibits "B" and "C" hereto, providing Fox-Warner with access to all film materials required for Delivery of the Picture.

15.	CALIFORNIA LAW:  This Agreement shall be construed and interpreted, and all questions as to performance determined in accordance with the laws of the State of California applicable to contracts executed and to be fully performed in the State of California.

 
16.	SEVERABILITY:  Nothing herein contained shall be construed to require the performance by either party of any act contrary to law or contrary to the rules and regulations of any guild having jurisdiction.  In the event of any conflict between any provision hereof and any law or governmental regulation or guild agreement, the latter shall prevail; but, in such event, the affected provisions of this Agreement shall be curtailed only to the extent necessary to bring them within the requirements of such law, regulation or guild agreement.
"""

print(context)

In [ ]:
question = """Highlight the parts (if any) of this contract related to "Governing Law" that should be reviewed by a lawyer. Details: Which state/country's law governs the interpretation of the contract?"""

In [ ]:
single_prediction(question,context)

#### Example 4

In [ ]:
context = """Hot Air Productions, Inc.
Mammoth Productions
8436 Wilshire Boulevard
Suite 730
Los Angeles, CA 90048

Re:	Rights Acquisition Agreement 
"Sex, Lies and Politcs"

Gentlemen:

This Cover Letter and its attachments constitute the Rights Acquisition Agreement which is entered into as of the 15th day of August, 2000, between F-W Entertainment N.V. ("Distributor") and Mammoth Productions ("Mammoth") and Hot Air Productions, Inc. ("Hot Air"). Distributor shall acquire from Hot Air, a motion picture produced by Hot Air under the conditions as spelled out in this Agreement.  The picture to be produced hereunder is "Sex, Lies And Politics" (the "Picture").  It is a provision of this Agreement, that upon Delivery of the Picture, Distributor shall pay to Hot Air an amount equal to the Acquisition Price as defined in paragraph 3., below. 

Whereas, Mammoth and Hot Air entered into a service agreement dated August 15, 2000 wherein Hot Air agreed to provide its services as the producer of the Picture, as an employee-for-hire and thereby fulfill Mammoth's production obligations with this Agreement.
"""

In [ ]:
question = '''Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract'''

In [ ]:
single_prediction(question,context)

#### Example 5

In [ ]:
context = """:  If Owner wishes to assign, sell, license, or otherwise exploit the Reserved Rights after the expiration of the Holdback Period, Owner will first notify Purchaser of Owner's intention to do so and will negotiate in good faith with Purchaser for a period of thirty (30) days with respect thereto.  If Purchaser is unable or unwilling to acquire the Reserved Rights or if the parties are unable to reach agreement upon the terms of such acquisition after such period of negotiation, then Owner may enter into negotiations with a third party or parties with respect to such rights.  However, that if Owner shall receive a bona fide offer from a third party which Owner wishes to accept, Owner may not conclude any agreement with such third party in relation to the assignment, sale, license, or exploitation of the Reserved Rights without first re-offering the same by notice in writing to Purchaser upon the same terms and conditions as are proposed by the third party.  Purchaser shall, within thirty (30) days after receipt of such notice from Owner, have the right to enter into an agreement with Owner within that thirty (30) day period on the same terms contained in such notice; provided that Purchaser shall not be obliged to meet any terms which cannot be met as easily by one person as another.  In the event that Purchaser shall not wish to enter into such an agreement, Owner shall have the right to accept the bona fide offer aforesaid.  Owner shall not enter into an agreement with a third party for the acquisition of the Reserved Rights on terms different from those submitted to Purchaser in connection with the bona fide offer aforesaid, unless and until Owner shall have notified Purchaser in writing of the new bona fide offer on such different terms and Purchaser shall have the opportunity to enter into an agreement with Owner in accordance with the procedure set forth herein with reference to the new bona fide offer. 
"""

In [ ]:
print(context)

In [ ]:
question = """Highlight the parts (if any) of this contract related to "Rofr/Rofo/Rofn" that should be reviewed by a lawyer. Details: Is there a clause granting one party a right of first refusal, right of first offer or right of first negotiation to purchase, license, market, or distribute equity interest, technology, assets, products or services?"""

In [ ]:
single_prediction(question,context)

## Long text

What do we do when the lenght of text is greater than 512 tokens?  Recall that most BERT-esque models can only accept 512 tokens at once. This means we'll have to split our input into chunks and each chunk must not exceed 512 tokens in total.

When working with Question Answering, it's crucial that each chunk follows this format:

[CLS] question tokens [SEP] context tokens [SEP]

So, we'll chunk the longer text, combine it with the question and evaluate all of the results.  Think of this like a slinding window approach for Q&A across the documents

In [24]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Note that we never want to truncate the question, only the context, else the only_second truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with return_overflowing_tokens=True and by passing the stride:

In [20]:
question = """Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract"""

In [21]:
context = """EXHIBIT 10.1   ENDORSEMENT AGREEMENT   This Endorsement Agreement (\"Agreement\") made October 30, 2017, between National Football League Alumni - Northern California Chapter (\"NFLA-NC\"), a charitable corporation organized under the laws of California, having its principal office at 1311 Madison Avenue, Redwood CA 94061; National Football League Alumni, Inc. (\"NFLA\"), a charitable corporation organized under the laws of Florida, having its principal office at 8000 Midlantic Drive, 130 S., Mount Laurel, NJ. 08054 and Food For Athletes, Inc. a corporation organized under the laws of California / Gridiron BioNutrients\u2122, a corporation organized under the laws of Nevada having their principal office(s) at 1119 West 1st Ave., STE G, Spokane, WA 99201 (collectively the \"Company\").   RECITALS   A. Whereas, NFLA is a nationwide group of former National Football League players, coaches, and other employees whose mission is to serve, assist and inform players and their families. The association offers a variety of medical, financial and social programs to help members lead healthy, productive and connected lives.    B. Whereas, NFLA-NC is a local Chapter of the NFLA and supports the organizations \"Caring for Kids\" initiative through fundraising for youth-related charities.    C. The Company desires to obtain the rights to use the Pro Football Legends Logo of the NFLA in connection with the advertisement and promotion of certain of its products. An image of the Pro Football Legends Logo is depicted in Exhibit A.    D. The NFLA agrees to license such rights to the Company.   In consideration of the matters described above, and of the mutual benefits and obligations set forth in this Agreement, the parties agree as follows:   SECTION ONE. DEFINITIONS   As used in this Agreement, the following terms shall be defined as follows:   A. \"Contract Period\" shall mean that period of time of three (3) years commencing on November 1st, 2017 and concluding November 2nd, 2020, unless terminated sooner or extended as provided in this Agreement.    B. \"Contract Territory\" shall mean worldwide.    C. \"Contract Year\" shall mean each of the consecutive 12-month periods beginning on the effective date of the Agreement of the Contract Period.   1\n\nSource: GRIDIRON BIONUTRIENTS, INC., 8-K, 12/6/2017\n\n\n\n\n\n      D. \"Gross Sales\" shall mean total revenues, under generally accepted accounting principles, from sales of the Licensed Products, but does not include any revenue from sales, use or other transaction taxes, duties, handling, graphics, embroidery or shipping.    E. \"Net Sales\" shall mean Gross Sales less Product returns, trade discounts, samples, allowances, value added services, markdowns, customer charge backs and liquidation sales (substantially discounted and out of ordinary distribution channel) of Licensed Products.    F. \"Licensed Products\" shall mean BlackMP Living Water, BlackMP Concentrate, Zezel Probiotic Water, Zayin Sports Water, Gridiron MVP\u2122 and Gridiron MVP\u2122 Concentrate using the Pro Football Legends Logo on the Licensed Products' affixed labels, hang-tags or packaging. Other products of the Company may be added to the list of Licensed Products during the Contract Period by written amendment to this Agreement. All amendments to this Agreement must be signed by all parties to this Agreement.    G. \"Products\" shall mean goods manufactured, distributed or otherwise sold by the Company.    H. \"Licensed Marks\" shall mean in connection with the rights and benefits granted to Company hereunder as set forth in General Terms. Company may utilize only the logo and other trademarks listed on Exhibit A (the \"Licensed Marks\") during the Term and within the Territory solely in connection with advertising and promotional materials that identify Company as a sponsor of the Pro Football Legends, provided that NFLA first approves all such uses in writing. Any use of the Licensed Marks will bear the trademark and/or copyright notices required by NFLA to facilitate its trademark protection program and will be a \"work made for hire\" for NFLA. All of Company's uses of the Licensed Marks shall inure to the benefit of the NFLA. After the expiration or termination of this Agreement, Company will refrain from further use of the Licensed Marks used pursuant to this Agreement. Company will not sublicense pass-through or otherwise grant to any third parties the rights granted to Company hereunder without the NFLA prior written consent, including but not limited to the right to use the Licensed Marks. Company acknowledges that this Agreement does not grant Company any rights with respect to any other NFLA Marks (defined below), the name, likeness, signature, or other attributes of any NFLA member or other individual, or any audio or video of any NFLA event. Company agrees that the quality of all services offered by Company under the Licensed Marks will conform to Licensor's written quality control standards and that Company will annually provide to NFLA samples of any advertising and marketing materials that use the Licensed Marks.    I. \"Trademark Protection\" for the purposes of this agreement, \"NFLA MARKS\" means the names, symbols, emblems, designs, and colors of the NFLA, including but not limited to the Licensed Marks. Company acknowledges and agrees that all right, title and interest in and to the NFLA marks belongs to the NFLA. Company agrees that NFLA marks possesses a special, unique and extraordinary character that makes difficult assessment of the monetary damages that would be sustained by their unauthorized use. Company recognizes that irreparable injury would be caused by unauthorized use of any of the NFLA marks, and agrees that injunctive and other equitable relief would be appropriate in the event of such unauthorized use, and that such remedy would not be exclusive of other legal remedies available to NFLA. Company recognizes that great value and goodwill associated with NFLA marks belongs to the NFLA and that the NFLA marks have secondary meaning.   2\n\nSource: GRIDIRON BIONUTRIENTS, INC., 8-K, 12/6/2017\n\n\n\n\n\n      J. \"NFLA Identification\" means the right to use, subject to the provisions of this Agreement, the NFLA name, and Pro Football Legends Logo and any other means of endorsement by the NFLA used in connection with the advertisement and promotion of the Company and the Licensed Products.    K. \"One (1) Unit\" shall represent $0.05 (1 Unit = $0.05USD) for purposes of defining the monetary donation allocation of the Company's sold products to the NFLA-NC, specific to the terms of this Agreement. A Licensed Product in no circumstance shall be valued at less than one (1) full Unit and under no circumstance shall a Unit be fractionalized (if required rounded up to the nearest whole number). SECTION TWO. GRANT OF RIGHTS   In consideration of the remuneration to be paid to the NFLA-NC pursuant to this Agreement, the NFLA grants to Company and to its authorized distributors and sublicenses the right and license during the Contract Period to use the NFLA Identification solely in connection with the advertisement, marketing and promotion of the Products within the Contract Territory as set forth in this Agreement. NFLA agrees not to grant the right to use the NFLAs Identification to anyone other than Company in connection with the advertisement and promotion of Products. It is understood that Company, its authorized distributors and sublicenses may not use the name of the NFLA in connection with any items for sale or resale, other than the Products as specified in this Agreement. The foregoing rights to use the name of the NFLA is limited to television, radio and print advertising, advertising published over the Internet (provided that such material is limited to advertising or Product promotion only), public relations and marketing materials, point-of-sale displays, free standing inserts, videos shown to customers and consumers, catalogs for customers and consumers, direct mail (including e-mail) and billboards. Company shall ensure that all uses of the name of the NFLA comply with applicable law.   SECTION THREE. PRIOR APPROVAL   Company agrees that no use of the name of the NFLA Identification nor any item used in connection with the name of NFLA Identification (including any Licensed Product) will be made under this Agreement unless and until the same is approved by the NFLA. The NFLA agrees that any material, advertising or otherwise, submitted for approval as provided in this section may be deemed by Company to have been approved under this section if the same is not disapproved in writing within ten (10) business days after receipt of the material. The NFLA agrees that it will reasonably cooperate with Company and that any material submitted under this section will not be unreasonably disapproved and, if it is disapproved, that Company will be advised of the specific grounds for disapproval. If Company desires immediate approval of advertising material, Company shall have the right to directly contact the NFLA's authorized agent to obtain such approval. Company agrees to protect, indemnify and hold harmless the NFLA and their authorized agents, or any of them, from and against any and all expenses, damages, claims, suits, actions, judgments and costs whatsoever, arising out of, or in any way connected with any advertising material furnished by, or on behalf of Company, except with respect to any inaccurate information furnished by them expressly for use in such advertising.   3\n\nSource: GRIDIRON BIONUTRIENTS, INC., 8-K, 12/6/2017\n\n\n\n\n\n      SECTION FOUR. REMUNERATION   In consideration of the endorsement rights granted under this Agreement, Company shall provide the following remuneration:   A. An initial one-time license fee of $35,000.00USD (Thirty-Five Thousand Dollars) payable to the NFLA-NC;    B. A one-time $10,000USD (Ten Thousand Dollars) promotional fee payable to NFLA. The $10,000 promotional fee shall be payable in four (4) quarterly payments beginning in 2018: Q1 ($2500); Q2 ($2500); Q3 ($2500); Q4 ($2500). Each payment shall be delivered by no later than the 10t h day of the first month of each quarter.    C. A *donation of $0.05 per Unit sold of Licensed Products within the Contract Territory payable to the **NFL Alumni Northern California Chapter. Donated amounts will be allocated and dispersed to the Northern California Chapter beginning on the first full quarter [three (3) month period] of the Agreement and continue on a quarterly basis thereafter for the term of this Agreement. Where the following per Unit conversion shall apply for the term of this Agreement:\n\n a. (1) Bottle of BlackMP LivingWater = 1 Unit\n\n b. (1 )  4oz  bo t t l e  o f  BlackMPConcentrate = 30 Units\n\n c. (1) Bottle of Zezel ProbioticWater = 1 Unit\n\n d. (1) Bottle of Zayin Sports Water = 1 Unit  e. (1) Bottle Gridiron MVP\u2122 Water= 1 Unit\n\n f. (1) 4oz bottle of Gridiron MVP\u2122Concentrate = 30 Units\n\n _____________    * The NFLA-NC will donate 15% of the above described proceeds to the NFLA.\n\n ** The Company will provide to the NFLA-NC upon request the most recent quarterly sales report of the Company'sLicensed Products.   D. Product Commitment. Up to Two-thousand (2,000) 4oz bottles of BlackMP Living Water Concentrate to be used as a \"value appeal\"  for  annual  membership renewal  (up to  a  $220,000 value)  a l located to  the  NFLA and provide a combination/assortment of bottle water to NFLA-NC for display and use though out the term of the Agreement at the NFLA-NC facilities. The Company agrees to pay for all shipping costs of the Products under the Product Commitment to the fulfillment center, currently Sharp Marketing, located in Fort Lauderdale, FL and to any subsequent fulfillment center that is contracted by the NFLA to provide distribution to existing and new NFLA members.    E. Affiliated Partnership Commitment. Company agrees to facilitate and provide in good-faith their affiliated partnership discount program benefits and access to applicable health and wellness research, information and protocols to NFLA members (Current partnership benefit includes providing NFLA members with a 15% discount on all Kraski's Nutrition Real Products For Real People); and    F. Marketing Commitment. Company agrees that it will continue in good-faith to produce and market Licensed Products in the same manner that it is currently producing and marketing such items as of September 2017, unless Company and the NFLA/NFLA-NC believe it is not commercially reasonable to continue to produce and market the Licensed Products.   4\n\nSource: GRIDIRON BIONUTRIENTS, INC., 8-K, 12/6/2017\n\n\n\n\n\n      SECTION FIVE. SERVICES OF COMPANY   A. Resource Call Center. For the term of this Agreement Company shall provide a call in center whereby NFLA members can call for information, ask questions, and consult with Company's staff on details and specifics of the Company's Products and replenishment program.   """

In [25]:
tokenized_example = tokenizer(
    question,
    context,
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride)

In [26]:
tokenized_example

{'input_ids': [[0, 18522, 6991, 5, 1667, 36, 1594, 143, 43, 9, 42, 1355, 1330, 7, 22, 19897, 43563, 10566, 113, 14, 197, 28, 7123, 30, 10, 2470, 4, 10574, 35, 20, 1248, 9, 5, 1355, 2, 2, 6725, 725, 8863, 2068, 158, 4, 134, 1437, 1437, 34588, 18347, 20944, 5680, 4629, 20944, 1437, 1437, 152, 4680, 27209, 1757, 8759, 6697, 19897, 43563, 8070, 156, 779, 389, 6, 193, 6, 227, 496, 3910, 815, 726, 28709, 111, 2874, 886, 8749, 6697, 12048, 250, 12, 6905, 16844, 10, 13615, 12772, 5798, 223, 5, 2074, 9, 886, 6, 519, 63, 5402, 558, 23, 508, 1225, 6370, 2666, 6, 1211, 1845, 5267, 361, 1749, 5606, 131, 496, 3910, 815, 726, 28709, 6, 603, 4, 6697, 12048, 250, 16844, 10, 13615, 12772, 5798, 223, 5, 2074, 9, 1261, 6, 519, 63, 5402, 558, 23, 38433, 4079, 462, 26970, 3936, 6, 8325, 208, 482, 5455, 22729, 6, 10585, 4, 321, 2940, 4283, 8, 3652, 286, 27417, 293, 6, 603, 4, 10, 12772, 5798, 223, 5, 2074, 9, 886, 1589, 22665, 11680, 12334, 43977, 44067, 13445, 6, 10, 12772, 5798, 223, 5, 2074, 9, 5077, 519,

In [27]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 384, 253]

In [28]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

<s>Highlight the parts (if any) of this contract related to "Agreement Date" that should be reviewed by a lawyer. Details: The date of the contract</s></s>EXHIBIT 10.1   ENDORSEMENT AGREEMENT   This Endorsement Agreement ("Agreement") made October 30, 2017, between National Football League Alumni - Northern California Chapter ("NFLA-NC"), a charitable corporation organized under the laws of California, having its principal office at 1311 Madison Avenue, Redwood CA 94061; National Football League Alumni, Inc. ("NFLA"), a charitable corporation organized under the laws of Florida, having its principal office at 8000 Midlantic Drive, 130 S., Mount Laurel, NJ. 08054 and Food For Athletes, Inc. a corporation organized under the laws of California / Gridiron BioNutrients™, a corporation organized under the laws of Nevada having their principal office(s) at 1119 West 1st Ave., STE G, Spokane, WA 99201 (collectively the "Company").   RECITALS   A. Whereas, NFLA is a nationwide group of former 

In [1]:
from predict import run_prediction
import json
import tqdm

In [2]:
with open('./data/CUADv1.json') as json_file:
    data = json.load(json_file)

In [3]:
questions = []
for i, q in enumerate(data['data'][0]['paragraphs'][0]['qas']):
    question = data['data'][0]['paragraphs'][0]['qas'][i]['question']
    questions.append(question)
contract = data['data'][0]['paragraphs'][0]['context']

In [4]:
with open('./temp/contract.txt', 'w') as f:
    f.write(' '.join(contract.split()))

In [5]:
[questions[3]]

['Highlight the parts (if any) of this contract related to "Effective Date" that should be reviewed by a lawyer. Details: The date when the contract is effective\xa0']

In [6]:
%%time 
out = run_prediction([questions[3]],contract,"./trained_models/roberta-base/")
print(out)

./trained_models/roberta-base/


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9686.61it/s]


[<transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dcf8>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dcc0>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2db38>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2d9e8>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2db00>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dc18>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dc50>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dba8>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dbe0>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2db70>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2dac8>, <transformers.data.processors.squad.SquadResult object at 0x7fcf36a2da58>, <transformers.data.processors.squad.SquadResult object at 0x7fcfe3b953c8>, <transformers.data.proce

In [40]:
import json

In [41]:
preds = json.load(open('./results/nbest.txt', 'rb'))

In [43]:
preds['0']

[{'text': 'The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter.',
  'probability': 0.7390376093749573,
  'start_logit': 9.529376983642578,
  'end_logit': 9.426068305969238},
 {'text': '7th day of September, 1999.',
  'probability': 0.2114331269146031,
  'start_logit': 8.802209854125977,
  'end_logit': 8.901795387268066},
 {'text': 'Unless earlier terminated otherwise provided therein, this Agreement, subject to the commencement date established in Section 1.3, shall be effective immediately.',
  'probability': 0.04378594324949106,
  'start_logit': 8.206672668457031,
  'end_logit': 7.922736644744873},
 {'text': 'this 7th day of September, 1999.',
  'probability': 0.005682502386584134,
  'start_logit': 5.18569278717041,
  'end_logit': 8.901795387268066},
 {'text': 'The',
  'probability': 6.064899567679408e-05,
  'start_logit': 9.529376983642578,
  'end_logit

In [16]:
%%time 
run_prediction(questions,contract,"./trained_models/roberta-base/")

./trained_models/roberta-base/


add example index and unique id: 100%|██████████| 41/41 [00:00<00:00, 34208.57it/s]


CPU times: user 1min, sys: 19 s, total: 1min 19s
Wall time: 2min 1s


OrderedDict([('0', 'DISTRIBUTOR AGREEMENT'),
             ('1', 'Distributor'),
             ('2', '7th day of September, 1999.'),
             ('3',
              'The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter.'),
             ('4',
              'The term of this Agreement shall be ten (10) years (the "Term") which shall commence on the date upon which the Company delivers to Distributor the last Sample, as defined hereinafter.'),
             ('5',
              'If Distributor complies with all of the terms of this Agreement, the Agreement shall be renewable on an annual basis for one (1) year terms for up to another ten (10) years on the same terms and conditions as set forth herein.'),
             ('6',
              'If Distributor complies with all of the terms of this Agreement, the Agreement shall be renewable on an annual basis for one (1

In [8]:
import transformers
transformers.__version__

'4.5.1'

In [ ]:
context = """CONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nprotection and deceptive trade practices, including any state anti-kickback/fraud and abuse related laws, all as amended from time to time.\n\n1.8 \"Business Day\" means each day of the week, excluding Saturday, Sunday or a day on which banking institutions in New York, New York, USA are closed.\n\n1.9 \"Calendar Quarter\" shall mean each successive period of three (3) calendar months commencing on January 1, April 1, July 1 and October 1, except that the first Calendar Quarter of the Term shall commence on the Effective Date and end on the day immediately prior to the first to occur of January 1, April 1, July 1 or October 1 after the Effective Date, and the last Calendar Quarter shall end on the last day of the Term.\n\n1.10 \"Calendar Year\" shall mean each successive period of twelve (12) months commencing on January 1 and ending on December 31, except that the first Calendar Year of the Term shall commence on the Effective Date and end on December 31 of the year in which the Effective Date occurs, and the last Calendar Year of the Term shall commence on January 1 of the year in which the Term ends and end on the last day of the Term.\n\n1.11 \"Claims\" shall mean all charges, complaints, actions, suits, proceedings, hearings, investigations, claims, demands, judgments, orders, decrees, stipulations or injunctions, in each case of a Third Party (including any Governmental Authority).\n\n1.12 \"Code\" shall mean the Code on Interactions with Healthcare Professionals promulgated by the Pharmaceutical Research and Manufacturers of America (PhRMA)/BIO, as it may be amended.\n\n1.13 \"Compensation Report\" shall have the meaning set forth in Section 4.2.2(b)"""

print(context)
#context = """"Exhibit 10.2\n\n______________________________________________________________________________\n\nCO-PROMOTION AGREEMENT\n\nby and between\n\nDOVA PHARMACEUTICALS, INC.\n\nand\n\nVALEANT PHARMACEUTICALS NORTH AMERICA LLC\n\nSeptember 26, 2018\n\n______________________________________________________________________________\n\nCONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nTABLE OF CONTENTS\n\nPage\n\nARTICLE 1 DEFINITIONS 1\n\nARTICLE 2 RIGHTS AND OBLIGATIONS 8\n\n2.1 Engagement; Grant of Rights. 8\n\n2.2 Retention of Rights. 9\n\n2.3 Non-Competition; Non-Solicitation. 9\n\n2.4 Dova Trademarks and Copyrights. 10\n\nARTICLE 3 JOINT STEERING COMMITTEE 11\n\n3.1 Formation of the JSC. 11\n\n3.2 Meetings and Minutes. 11\n\n3.3 Purpose of the JSC. 11\n\n3.4 Decision Making. 13\n\n3.5 Marketing Sub-Committee. 13\n\nARTICLE 4 VALEANT ACTIVITIES FOR THE PRODUCT 14\n\n4.1 Valeant Activities. 14\n\n4.2 Detailing. 15\n\n4.3 Compliance with Applicable Law. 17\n\n4.4 Field Force Personnel Training; Product Materials. 19\n\n4.5 Provisions Related to Field Force Personnel. 21\n\n4.6 Responsibility for Valeant Activity Costs and Expenses. 22\n\n4.7 Data Sharing. 22\n\nARTICLE 5 REGULATORY, SAFETY AND SURVEILLANCE, COMMERCIAL MATTERS 23\n\n5.1 Dova Responsibility. 23\n\n5.2 Valeant Involvement. 23\n\n5.3 Inspections. 23\n\n5.4 Pharmacovigilance. 24\n\n5.5 Unsolicited Requests for Medical Information. 24\n\n5.6 Recalls and Market Withdrawals. 25\n\n5.7 Certain Reporting Responsibilities. 25\n\n5.8 Booking of Sales Revenues. 25\n\n5.9 Returns. 25\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\ni\n\nCONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nTABLE OF CONTENTS (continued)\n\n5.10 Manufacturing; Distribution; Marketing. 25\n\nARTICLE 6 FINANCIAL PROVISIONS 26\n\n6.1 Promotion Fee. 26\n\n6.2 Milestone Payment. 27\n\n6.3 Reports; Payments. 27\n\n6.4 Taxes. 28\n\n6.5 Determination of Specialty. 29\n\nARTICLE 7 AUDIT RIGHTS 30\n\n7.1 Recordkeeping. 30\n\n7.2 Valeant Rights. 30\n\n7.3 Dova Rights. 31\n\nARTICLE 8 INTELLECTUAL PROPERTY 32\n\n8.1 Ownership of Intellectual Property. 32\n\n8.2 Title to Trademarks and Copyrights. 32\n\n8.3 Protection of Trademarks and Copyrights. 32\n\n8.4 Disclosure of Know-How. 33\n\nARTICLE 9 CONFIDENTIALITY 33\n\n9.1 Confidential Information. 33\n\n9.2 Public Announcements. 34\n\nARTICLE 10 REPRESENTATIONS AND WARRANTIES; ADDITIONAL COVENANTS 35\n\n10.1 Representations and Warranties of Dova. 35\n\n10.2 Representations and Warranties of Valeant. 37\n\n10.3 Disclaimer of Warranty. 38\n\n10.4 Additional Covenants. 39\n\nARTICLE 11 INDEMNIFICATION; LIMITATIONS ON LIABILITY 39\n\n11.1 Indemnification by Dova. 39\n\n11.2 Indemnification by Valeant. 39\n\n11.3 Indemnification Procedures. 40\n\n11.4 Limitation of Liability. 40\n\n11.5 Insurance. 40\n\nARTICLE 12 TERM AND TERMINATION 41\n\n12.1 Term. 41\n\n12.2 Early Termination for Cause. 41\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\n12.3 Other Early Termination. 42\n\n12.4 Effects of Termination. 42\n\n12.5 Tail Period. 42\n\nii\n\nCONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nTABLE OF CONTENTS (continued)\n\n12.6 Survival. 43\n\nARTICLE 13 MISCELLANEOUS 43\n\n13.1 Force Majeure. 43\n\n13.2 Assignment. 43\n\n13.3 Severability. 44\n\n13.4 Notices. 44\n\n13.5 Governing Law. 45\n\n13.6 Dispute Resolution. 45\n\n13.7 Waiver of Jury Trial. 45\n\n13.8 Entire Agreement; Amendments. 46\n\n13.9 Headings. 46\n\n13.10 Independent Contractors. 46\n\n13.11 Third Party Beneficiaries. 46\n\n13.12 Waiver. 46\n\n13.13 Cumulative Remedies. 46\n\n13.14 Waiver of Rule of Construction. 46\n\n13.15 Use of Names. 46\n\n13.16 Further Actions and Documents. 47\n\n13.17 Certain Conventions. 47\n\n13.18 Counterparts. 47\n\niii\n\nCONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nCO-PROMOTION AGREEMENT\n\nThis Co-Promotion Agreement (this \"Agreement\") is entered into and dated as of September 26, 2018 (the \"Effective Date\") by and between Dova Pharmaceuticals, Inc., a Delaware corporation (\"Dova\"), and Valeant Pharmaceuticals North America LLC, a Delaware limited liability company (\"Valeant\"). Dova and Valeant are each referred to individually as a \"Party\" and together as the \"Parties\".\n\nRECITALS\n\nWHEREAS, Dova has developed and has rights to market and sell the Product (as defined below) in the Territory;\n\nWHEREAS, the Parties believe that it would be mutually beneficial to collaborate on promotional activities for the Product and, accordingly, Dova desires that Valeant conduct certain promotional activities, and Valeant desires to conduct such activities, for the Product in the Territory;\n\nNOW, THEREFORE, in consideration of the following mutual promises and obligations, and for other good and valuable consideration the adequacy and sufficiency of which are hereby acknowledged, the Parties agree as follows:\n\nCONFIDENTIAL TREATMENT HAS BEEN REQUESTED FOR PORTIONS OF THIS EXHIBIT. THE COPY FILED HEREWITH OMITS THE INFORMATION SUBJECT TO A CONFIDENTIALITY REQUEST. OMISSIONS ARE DESIGNATED [***]. A COMPLETE VERSION OF THIS EXHIBIT HAS BEEN FILED SEPARATELY WITH THE SECURITIES AND EXCHANGE COMMISSION.\n\nSource: DOVA PHARMACEUTICALS INC., 10-Q, 11/8/2018\n\n\n\n\n\nARTICLE 1   DEFINITIONS\n\n1.1 \"Act\" shall mean the Federal Food, Drug and Cosmetic Act, 21 U.S.C. \u00a7 301 et seq., as it may be amended from time to time, and the regulations promulgated thereunder.\n\n1.2 \"Adverse Event\" shall mean any untoward medical occurrence in a patient or clinical investigation subject who is administered the Product, but which does not necessarily have a causal relationship with the treatment for which the Product is used. An \"Adverse Event\" can include any unfavorable and unintended sign (including an abnormal laboratory finding), symptom or disease temporally associated with the use of the Product, whether or not related to the Product. A pre-existing condition that worsened in severity after administration of the Product would be considered an \"Adverse Event\".\n\n1.3 \"Affiliate\" shall mean, with respect to any Person, any other Person that directly or indirectly controls, is controlled by or is under common control with such Person. A Person shall be deemed to control another Person if such Person possesses the power to direct or cause the direction of the management, business and policies of such Person, whether through the ownership of fifty percent (50%) or more (or such lesser percentage which is the maximum allowed to be owned by a foreign corporation in a particular jurisdiction) of the voting securities of such Person, by contract or otherwise.\n\n1.4 \"Agreement\" shall have the meaning set forth in the preamble to this Agreement.\n\n1.5 \"Alliance Managers\" shall have the meaning set forth in Section 4.1.4.\n\n1.6 \"Alternate Product\" shall mean a pharmaceutical product that is commercialized by Valeant or its Affiliates in the Territory and that is part of the Salix business segment of Valeant's parent company, Bausch Health Companies, Inc. (or, in the event that such business segments are restructured, that is part of the Salix business unit), and which product is complementary to the Product with regard to Target Professionals in the Specialty.\n\n1.7 \"Applicable Laws\" shall mean all applicable statutes, ordinances, regulations, codes, rules, or orders of any kind whatsoever of any Governmental Authority in the Territory pertaining to any of the activities and obligations contemplated by this Agreement, including, as applicable, the Act, the Generic Drug Enforcement Act of 1992 (21 U.S.C. \u00a7 335a et seq.), the Anti- Kickback Statute (42 U.S.C. \u00a7 1320a-7b et seq.)"""
# 1. TOKENIZE THE INPUT
# note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for 
# exploration but you cannot feed that into a model. 
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. OBTAIN MODEL SCORES
# the AutoModelForQuestionAnswering class includes a span predictor on top of the model. 
# the model returns answer start and end scores for each word in the text
output = model(**inputs)
answer_start_scores, answer_end_scores = model(**inputs,return_dict=False)
answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

# 3. GET THE ANSWER SPAN
# once we have the most likely start and end tokens, we grab all the tokens between them
# and convert tokens back to words!
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

In [ ]:
answer_start_scores

In [ ]:
answer_end_scores

In [14]:
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

In [ ]:
SquadExample()